# Welcome to my Histopathologic Cancer Detection Neural Network (Notebook)

### Kaggle code


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



### Checking the library versions:

In [4]:
import tensorflow as tf
print("tensorflow:", tf.__version__)

# import keras
# print("keras:", keras.__version__)

# import kerastuner as kt
# print("kerastuner:", kt.__version__)

# import keras_tuner as kt2
# print("keras_tuner:", kt2.__version__)

import platform
print("Python:", platform.python_version())

import numpy as np
print("numpy:", np.__version__)

import pandas as pd
print("pandas:", pd.__version__)

import sklearn
print("sklearn version:", sklearn.__version__)

import sklearn
print("sklearn path:", sklearn.__path__)

import matplotlib
print("matplotlib:", matplotlib.__version__)

import seaborn as sns
print("seaborn:", sns.__version__)

# Tensorflow: 2.15.0
# kerastuner: 1.0.5
# keras_tuner: 1.3.5
# Python: 3.10.11
# numpy: 1.24.3
# pandas: 2.1.4
# sklearn version: 1.2.2
# sklearn path: ['c:\\Users\\Micha\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\sklearn']
# matplotlib: 3.8.2
# seaborn: 0.13.0



tensorflow: 2.15.0
Python: 3.10.11
numpy: 1.24.3
pandas: 2.1.4
sklearn version: 1.2.2
sklearn path: ['c:\\Users\\Micha\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\sklearn']
matplotlib: 3.8.2
seaborn: 0.13.0


### Set Global random seed to make sure we can replicate any model that we create (no randomness)

In [5]:
import random
import tensorflow as tf
import numpy as np
import os



np.random.seed(42)
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

os.environ['TF_DETERMINISTIC_OPS'] = '1'

## Image recognition pre-processing practices:

1. **Rescaling**: Image pixel values usually range from 0 to 255. Rescaling these values to a range of 0 to 1 by dividing each pixel by 255 is a common practice. This helps to stabilize and speed up the learning process.

2. **Resizing**: Deep learning models require the input dimensions to be uniform. Resizing all images to a predetermined size is essential, especially if the original dataset contains images of various dimensions.

3. **Normalization**: Beyond just rescaling, you might want to normalize the image data. This can include subtracting the mean and dividing by the standard deviation across each channel. If you have a pre-trained model, you would use the normalization statistics (mean and standard deviation) from the dataset on which the model was trained.

4. **Data Augmentation**: To increase the diversity of your dataset and prevent overfitting, you can apply random transformations like rotation, shifting, flipping, zooming, and shearing. These transformations generate new training samples from the original ones by altering them slightly.

5. **Color Space Conversions**: Sometimes, converting images to different color spaces (e.g., from RGB to grayscale, HSV, LAB, etc.) can help the model learn more robust features, depending on the task.

6. **Image Denoising**: If the images are noisy, applying denoising algorithms can help to remove noise and improve model accuracy.

7. **Edge Detection**: In certain applications, particularly those involving shape analysis, edge detection filters may be applied to highlight the edges within images.

8. **Masking and Cropping**: If there are regions in the images that are not relevant to the analysis, you might want to mask or crop these regions to focus the model on the important parts of the image.

9. **Histogram Equalization**: This can enhance the contrast in images, which can be beneficial if you have a dataset with varying lighting conditions.

10. **Centering and Standardization**: Similar to normalization, centering the data by subtracting the mean image (computed over the training set) and standardizing, so the variance of the pixels is reduced, can be beneficial.

11. **Handling Class Imbalance**: If your dataset has a class imbalance, techniques such as class weighting, oversampling the minority class, or undersampling the majority class can be considered.

In practice, preprocessing steps are often determined experimentally. You might start with a simple preprocessing pipeline (like just rescaling and resizing) and then iteratively add steps that improve your model performance. It's also important to note that if you're using a pre-trained model, you should preprocess your data in the same way the original model was trained.

### Check to see if each image has the same dimensions since that's important for data preprocessing

In [6]:
# from PIL import Image
# import os

# def check_image_dimensions(directory):
#     image_sizes = set()
#     for img_name in os.listdir(directory):
#         img_path = os.path.join(directory, img_name)
#         with Image.open(img_path) as img:
#             # Get image size
#             size = img.size
#             image_sizes.add(size)
            
#             # # If more than one size is found, we can stop checking
#             # if len(image_sizes) > 1:
#             #     break
    
#     if len(image_sizes) == 1:
#         print(f"For the {directory} directory, all images are of the same dimension: {image_sizes.pop()}")
#     else:
#         print(f"For the {directory} directory, different dimensions found: {image_sizes}")

# # Use it on the train and test data only if this code segment was never ran in this coding session:
# if 'checked_image_dimensions' not in globals():
#     # Use it on the train and test data:
#     check_image_dimensions('train')
#     check_image_dimensions('test')
#     checked_image_dimensions = True

# # For the train directory, all images are of the same dimension: (96, 96)
# # For the test directory, all images are of the same dimension: (96, 96)


### If you need to, call the resize_images functions to ensure each image is the same dimension but make sure you are not distorting the images. In order to do this, you need to make sure all the original images have the same aspect ratios

Just a warning: this will resize the original images. A better way to do this is in-memory processing, where you only save images in the disk for temporary use, like a variable in python. So, you can just create the function, give it to the TensorFlow ImageDataGenerator, and then it will call the resize function on all the images

In [7]:
from PIL import Image
import os
 

def resize_images(directory, size=(128, 128)): 
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        with Image.open(img_path) as img:
            new_img = img.resize(size)
            new_img.save(img_path)

# Use it on the train and test data if needed, and change the size argument as you need:
            
# resize_images('train', size=(128, 128))
# resize_images('test', size=(128, 128))

### Split the data into a training, validation, testing sets 
Make sure to do this before using data augmentation like ImageDataGenerator(). It's hard to split the data into train-validation-test after using ImageDataGenerator()

In [8]:
from sklearn.model_selection import train_test_split

# Set the base directory to the current directory
base_dir = ''

# Directory for train
train_dir = os.path.join(base_dir, 'train')

# Load the labels
labels = pd.read_csv(os.path.join(base_dir, 'train_labels.csv'))

# Convert the 'label' column to strings
labels['label'] = labels['label'].astype(str)

# Add the full path to the image files, and create a new column called "path" inside the label dataframe to store these paths to images
labels['path'] = labels['id'].apply(lambda x: os.path.join(train_dir, f"{x}.tif"))

# Split the labels dataframe into train, validation, and test sets into a 70/15/15 ratio
train_df, test_df = train_test_split(labels, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

# Save the DataFrames to CSV files
train_df.to_csv('training_df_labels.csv', index=False)
val_df.to_csv('valid_df_labels.csv', index=False)
test_df.to_csv('testing_df_labels.csv', index=False)

# Now we have a dataframe for train, val, and test which contains the data of their path, label, and id

### Use Keras ImageDataGenerator() on Train/Validation/Test split and also crop 32x32px center
The ImageDataGenerator not only helps you load images from the disk but also allows you to perform **data augmentation**, which is a technique to increase the diversity of your training set by applying random transformations (like rotation, zoom, flips, etc.) to the images. This is very useful to prevent overfitting and helps the model generalize better.

make sure to change the "target_size" argument of the train_datagen.flow_from_dataframe() function as needed

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


#This is a function to crop the image to focus on the 32x32px center of the image. We will call this function in the ImageDataGenerator() function
def crop_center(img): 
    y, x, _ = img.shape
    startx = x//2 - (32//2)
    starty = y//2 - (32//2)    
    return img[starty:starty+32, startx:startx+32, :]



# Creating an instance of the ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale the image pixel values to [0,1]
    preprocessing_function = crop_center  # call the crop function on each image

    # Potential data augmentation techniques that won't affect the 32x32px center
    #brightness_range=[0.8, 1.2], 
    #channel_shift_range=20, 

    # I removed these transformations for the data augmentation since this project involves detecting tumor tissue in the center 32x32px region so I can't be doing zooming and other transformations for this project specifically

    # rotation_range=40,  # Random rotations
    # width_shift_range=0.2,  # Random horizontal shifts
    # height_shift_range=0.2,  # Random vertical shifts
    # shear_range=0.2,  # Shear transformations
    # zoom_range=0.2,  # Random zoom
    # horizontal_flip=True,  # Random horizontal flips
    # fill_mode='nearest'  # Strategy for filling in new pixels
)

val_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function = crop_center)  # call the crop function on each image
test_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function = crop_center) # call the crop function on each image



# Flow from dataframe method to load images using the dataframe
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df, # Use the training dataframe (with labels, id, and paths)
    x_col='path',
    y_col='label',
    target_size=(32, 32),  # The dimensions to which all images found will be resized. Change this as needed
    color_mode='rgb',
    class_mode='binary', # means that the labels are binary labels
    batch_size=32,
    shuffle=True, # This might introduce randomness if set to true, but if it's false, the it might lead to overfitting. So it's best to just save the neural network to ensure no randomness
    seed=42
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df, # Use the validation dataframe (with labels, id, and paths)
    x_col='path',
    y_col='label',
    target_size=(32, 32),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False,
    seed=42
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df, # Use the testing dataframe (with labels, id, and paths)
    x_col='path',
    y_col='label',
    target_size=(32, 32),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False,
    seed=42
)



# After setting this up, you can use train_generator as the input to the fit or fit_generator method of your Keras model, 
# which will load images in batches and train your model on them.





Found 154017 validated image filenames belonging to 2 classes.
Found 33004 validated image filenames belonging to 2 classes.
Found 33004 validated image filenames belonging to 2 classes.


### If you want, you can check the image shape and see a visualization of the pictures below:

In [10]:
# import matplotlib.pyplot as plt

# # Get a batch of images
# images, labels = next(train_generator)

# # The images should now be a numpy array. Check its shape:
# print(images.shape)  # Should be (batch_size, target_size[0], target_size[1], 3)

# # Plot the first few images
# for i in range(5):  # Change this value to see more images
#     plt.figure(figsize=(5, 5))
#     plt.imshow(images[i])
#     plt.title(f'Label: {labels[i]}') 
#     plt.show()

### Here is another function which is able to crop images but you have to manually call this function on each image in order to crop, so I just used the ImageDataGenerator() method instead

In [11]:
from PIL import Image
import matplotlib.pyplot as plt

def crop_center(img):
    width, height = img.size
    new_width, new_height = 32, 32

    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2

    return img.crop((left, top, right, bottom))


#The code below is for you to visually see the cropped images

# # Get a batch of images
# images, labels = next(train_generator)

# # The images should now be a numpy array. Check its shape:
# print(images.shape)  # Should be (batch_size, target_size[0], target_size[1], 3)

# # Crop and plot the first few images
# for i in range(5):  # Change this value to see more images
#     img = Image.fromarray((images[i] * 255).astype(np.uint8))  # Convert to PIL Image
#     cropped_img = crop_center(img)
#     plt.figure(figsize=(5, 5))
#     plt.imshow(cropped_img)
#     plt.title(f'Label: {labels[i]}') 
#     plt.show()


### Use ImageDataGenerator on the actual test data (from the test directory, not the testing data from the train/valid/test split) 

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the base directory to the current directory
base_dir = ''

# Directory for test
test_dir = os.path.join(base_dir, 'test')

# Get the list of test image filenames
test_filenames = os.listdir(test_dir)

# Create a DataFrame with 'id' and 'path' columns
df_test = pd.DataFrame({
    'id': [filename.split('.')[0] for filename in test_filenames],
    'path': [os.path.join(test_dir, filename) for filename in test_filenames]
})



#This is a function to crop the image to focus on the 32x32px center of the image. We will call this function in the ImageDataGenerator() function
def crop_center(img): 
    y, x, _ = img.shape
    startx = x//2 - (32//2)
    starty = y//2 - (32//2)    
    return img[starty:starty+32, startx:startx+32, :]

# Create a data generator for the test data
real_test_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function = crop_center)

real_test_generator = real_test_datagen.flow_from_dataframe(
        dataframe = df_test,
        x_col="path",
        y_col=None,  # We don't have labels for the test data
        target_size=(32, 32),
        batch_size=32, # Change the batch size as needed
        class_mode=None,  # We don't have labels for the test data
        color_mode = "rgb",
        shuffle=False)

Found 57458 validated image filenames.


## Now, it's time to create my first model. This is Model 1

In [13]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# # 1. Define the model architecture
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Flatten(),
#     Dense(64, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])

# #2. Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# #3. Fit the model
# model.fit(train_generator, validation_data=val_generator, epochs=10)

# #4. Evaluate the model
# loss, accuracy = model.evaluate(test_generator)
# print('Test accuracy:', accuracy)

# #Took almost 23 min

1. `Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3))`: This line creates a 2D convolution layer. Convolution layers are the major building blocks used in convolutional neural networks. A convolution layer transforms an input volume into an output volume of different size, as specified by the parameters of the layer. In this case, the layer will output 32 different feature maps, each one representing a different learned feature. The `(3, 3)` parameter specifies the size of the filters that will be learned, and `relu` is the activation function that will be applied element-wise to the output. The `input_shape=(32, 32, 3)` parameter specifies the shape of the input data: images of size 32x32 pixels with 3 color channels (red, green, blue).

2. `MaxPooling2D((2, 2))`: This line creates a max pooling layer, which is used to reduce the spatial dimensions of the output volume from the previous layer. It does this by taking the maximum value over a 2x2 window. This helps to make the model more translation invariant and to reduce computation.

3. `Conv2D(64, (3, 3), activation='relu')`: This is another convolution layer, similar to the first one. This layer will learn 64 filters. The size of the filters is again 3x3 pixels, and the activation function is ReLU.

4. `MaxPooling2D((2, 2))`: This is another max pooling layer, similar to the first one. It again reduces the spatial dimensions of the output volume from the previous layer.

5. `Flatten()`: This layer flattens the output from the previous layer into a one-dimensional vector. This is necessary because the next layer (a dense layer) expects its input to be a vector, not a multi-dimensional array.

6. `Dense(64, activation='relu')`: This is a fully connected layer, also known as a dense layer. Each neuron in a dense layer receives input from all the neurons in the previous layer, hence they are "fully connected". This layer has 64 neurons and uses the ReLU activation function.

7. `Dense(1, activation='sigmoid')`: This is the output layer of the model. It's another dense layer, and it has just one neuron because this is a binary classification problem (assuming your labels are 0 and 1). The sigmoid activation function is used to squash the output of the neuron to a value between 0 and 1, representing the probability that the image belongs to class 1.

### Displaying information about Model 1

In [14]:
# # Model Summary:
# model.summary()


# # Get model configuration
# config = model.get_config()
# print(config)


# # Get model weights
# weights = model.get_weights()
# for i, weight in enumerate(weights):
#     print(f"Weights of layer {i}: {weight}")



# # Plot model architecture (visual representation)
# from tensorflow.keras.utils import plot_model
# plot_model(model, to_file='model1_plot.png', show_shapes=True, show_layer_names=True)   

### Submitting Model 1

In [15]:
# import pandas as pd
# from tensorflow.keras.models import load_model

# # Make predictions
# predictions = model.predict(real_test_generator, steps=len(real_test_generator), verbose=1)

# model.save("model1_saved.h5")

# # Get filenames (ordered list of image file names)
# filenames = real_test_generator.filenames
# print(filenames[:5])

# # Get the actual predictions, not the probabilities
# # If your model is a binary classifier, this will convert the probabilities into class predictions
# predicted_classes = [1 if prob > 0.5 else 0 for prob in predictions]

# # Create a DataFrame with filenames and predicted classes
# submission_df = pd.DataFrame({
#     'id': [os.path.basename(filename).split('.')[0] for filename in filenames],  # Extract the id from the filename
#     'label': predicted_classes
# })

# # Save DataFrame to csv
# submission_df.to_csv('submission.csv', index=False)
# print("Your submission was successfully saved!")

# # Took 51 seconds

### Making Model 3 using Keras Tuner for hyperparameter tuning

When tuning a Convolutional Neural Network (CNN) using Keras Tuner, there are several hyperparameters that are commonly experimented with:

1. **Number of Convolutional Layers:** The number of convolutional layers in the network can significantly affect the model's performance. More layers allow the model to learn more complex features, but can also lead to overfitting and longer training times.

2. **Number of Filters in Convolutional Layers:** The number of filters in each convolutional layer is another important hyperparameter. More filters allow the model to learn more complex features, but can also lead to overfitting and longer training times.

3. **Kernel Size in Convolutional Layers:** The size of the kernels used in the convolutional layers can affect the scale of the features that the model can learn. Smaller kernels allow the model to learn more local features, while larger kernels allow it to learn more global features.

4. **Pooling Size in Pooling Layers:** The size of the pooling operation can affect the amount of down-sampling performed in the network. Larger pooling sizes result in more down-sampling, which can help to reduce overfitting and computational cost, but can also cause the model to lose important information.

5. **Number of Units in Dense Layers:** The number of units in the dense layers can affect the capacity of the model. More units allow the model to learn more complex representations, but can also lead to overfitting and longer training times.

6. **Learning Rate:** The learning rate controls the step size during optimization. A learning rate that is too high can cause the model to converge too quickly to a suboptimal solution, while a learning rate that is too low can cause the model to converge too slowly.

7. **Optimizer:** Different optimizers can converge at different rates. Common choices include SGD, Adam, and RMSprop.

8. **Activation Functions:** The choice of activation function can affect the model's ability to learn complex patterns. Common choices include ReLU, sigmoid, and tanh.

Remember, the best hyperparameters can vary depending on the specific problem and dataset, so it's always a good idea to experiment with different values.

In [16]:
# from tensorflow import keras
# from tensorflow.keras.models import Sequential
# from kerastuner.tuners import Hyperband
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# from tensorflow.keras.callbacks import EarlyStopping

# def build_model(hp):
#     model = Sequential()
    
#     # Tune the number of filters in the first Conv2D layer
#     hp_filters = hp.Int('conv_1_filters', min_value = 32, max_value = 128, step = 16)
#     model.add(Conv2D(filters = hp_filters, kernel_size = (3,3), activation = 'relu', input_shape = (32,32,3)))
    
#     model.add(MaxPooling2D(pool_size=(2,2)))
    
#     for i in range(hp.Int('num_layers', 2, 5)):
#         model.add(Conv2D(filters = hp.Int(f'conv_{i+2}_filters', min_value = 32, max_value = 128, step = 16), 
#                                       kernel_size = hp.Choice(f'conv_{i+2}_kernel', values = [2,3]), 
#                                       activation = 'relu'))
#         model.add(MaxPooling2D(pool_size=hp.Choice(f'pool_{i+2}_size', values = [2,3])))
    
#     model.add(Flatten())
    
#     model.add(Dense(units = hp.Int('dense_1_units', min_value = 32, max_value = 512, step = 32), activation = 'relu'))
    
#     model.add(Dense(1, activation='sigmoid'))

#     # Tune the learning rate for the optimizer 
#     hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

#     model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
#                 loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
#                 metrics = ['accuracy'])

#     return model


# # Add the callback to the tuner
# tuner = Hyperband(
#     build_model,
#     objective='val_accuracy',
#     executions_per_trial=2,
#     directory='directory_for_model2',
#     project_name='keras_tuner_model2',
#     overwrite=False  # Enable resuming from checkpoints. If you set it equal to false, then the tuner will resume from the last checkpoint if one exists
# )


# # Define the early stopping callback
# early_stopping_callback = EarlyStopping(
#     monitor='val_loss',  # Monitor validation loss
#     patience=3,  # Number of epochs with no improvement after which training will be stopped
#     restore_best_weights=True  # Restore model weights from the epoch with the best value
# )


# # Perform the hyperparameter search
# tuner.search(train_generator, 
#              epochs=5, 
#              validation_data=val_generator,
#              callbacks=[early_stopping_callback])  # Add the callback here

# # Get the best model
# best_model = tuner.get_best_models(num_models=1)[0]


# #Print summary of the search space for the hyperparameters
# tuner.search_space_summary()

# # Get the best model
# best_model = tuner.get_best_models(num_models=1)[0]

# # Trial 10 Complete [00h 00m 00s]

# # Best val_accuracy So Far: 0.8330959677696228
# # Total elapsed time: 00h 39m 44s

# # Search: Running Trial #11

# # Value             |Best Value So Far |Hyperparameter
# # 112               |112               |conv_1_filters
# # 4                 |2                 |num_layers
# # 96                |80                |conv_2_filters
# # 3                 |2                 |conv_2_kernel
# # 3                 |2                 |pool_2_size
# # 80                |112               |conv_3_filters
# # 3                 |3                 |conv_3_kernel
# # 3                 |2                 |pool_3_size
# # 160               |224               |dense_1_units
# # 0.0001            |0.001             |learning_rate
# # 112               |32                |conv_4_filters
# # 3                 |3                 |conv_4_kernel
# # 3                 |2                 |pool_4_size
# # 2                 |2                 |tuner/epochs
# # 0                 |0                 |tuner/initial_epoch
# # 4                 |4                 |tuner/bracket
# # 0                 |0                 |tuner/round


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

best_model = Sequential([
    Conv2D(112, (3,3), activation='relu', input_shape=(32,32,3)),
    MaxPooling2D((2,2)),
    Conv2D(80, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(112, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(224, activation='relu'),
    Dense(1, activation='sigmoid')
])

best_model.compile(optimizer = Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

def combine_generators(gen1, gen2):
    while True:
        yield next(gen1)
        yield next(gen2)

combined_generator = combine_generators(train_generator, val_generator) # I combined both the training and validation data so I can train the model on both, and then used the testing data as the validation data

best_model.fit(combined_generator, steps_per_epoch=len(train_generator) + len(val_generator), validation_data = test_generator, epochs=5)

# Took 17 minutes

Epoch 1/5
5846/5846 [==============================] - 232s 40ms/step - loss: 0.4537 - accuracy: 0.7917 - val_loss: 0.4027 - val_accuracy: 0.8196
Epoch 2/5
5846/5846 [==============================] - 206s 35ms/step - loss: 0.3891 - accuracy: 0.8278 - val_loss: 0.3759 - val_accuracy: 0.8323
Epoch 3/5
5846/5846 [==============================] - 188s 32ms/step - loss: 0.3684 - accuracy: 0.8379 - val_loss: 0.3733 - val_accuracy: 0.8338
Epoch 4/5
5846/5846 [==============================] - 212s 36ms/step - loss: 0.3531 - accuracy: 0.8463 - val_loss: 0.3672 - val_accuracy: 0.8404
Epoch 5/5
5846/5846 [==============================] - 216s 37ms/step - loss: 0.3425 - accuracy: 0.8520 - val_loss: 0.3551 - val_accuracy: 0.8464


### Displaying Information about Model 2

In [ ]:
# # Model Summary:
# best_model.summary()


# # Get model configuration
# config = best_model.get_config()
# print(config)


# # Get model weights
# weights = best_model.get_weights()
# for i, weight in enumerate(weights):
#     print(f"Weights of layer {i}: {weight}")



# # Plot model architecture (visual representation)
# from tensorflow.keras.utils import plot_model
# plot_model(best_model, to_file='model2_plot.png', show_shapes=True, show_layer_names=True)   

### Submitting Model 2. Score: 82.91

In [21]:
import pandas as pd
from tensorflow.keras.models import load_model

#Use the model from the keras tuner
model2 = best_model

# Make predictions
predictions = model2.predict(real_test_generator, steps=len(real_test_generator), verbose=1)

model2.save("model2_saved.h5")

# Get filenames (ordered list of image file names)
filenames = real_test_generator.filenames
print(filenames[:5])

# Get the actual predictions, not the probabilities
# If your model is a binary classifier, this will convert the probabilities into class predictions
predicted_classes = [1 if prob > 0.5 else 0 for prob in predictions]

# Create a DataFrame with filenames and predicted classes
submission_df = pd.DataFrame({
    'id': [os.path.basename(filename).split('.')[0] for filename in filenames],  # Extract the id from the filename
    'label': predicted_classes
})

# Save DataFrame to csv
submission_df.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

# Took 1 minute

1796/1796 [==============================] - 60s 33ms/step


c:\Users\Micha\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['test\\00006537328c33e284c973d7b39d340809f7271b.tif', 'test\\0000ec92553fda4ce39889f9226ace43cae3364e.tif', 'test\\00024a6dee61f12f7856b0fc6be20bc7a48ba3d2.tif', 'test\\000253dfaa0be9d0d100283b22284ab2f6b643f6.tif', 'test\\000270442cc15af719583a8172c87cd2bd9c7746.tif']
Your submission was successfully saved!


### 1. Maybe try GrayScale conversation
### 2. Try Image cropping for 32x32px or 33x33px or no image cropping at all
### 3. Try histogram equalization (part of data preprocessing)
### 4. Find a way to make the images less blurry or find a way to make it not lose any pixels since each pixel is important
### 5. Think about adding regularization into Keras tuner and maybe trying out different activation functions but adding activation functions will increase the run time a lot